In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/edaic-aus-pose-gaze/312_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/451_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/338_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/325_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/673_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/308_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/449_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/314_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/633_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/306_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/448_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/344_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/337_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-aus-pose-gaze/467_OpenFace2.1.0_Pose_gaze_AUs.csv
/kaggle/input/edaic-

In [2]:
import pandas as pd
import os


def split_openface_by_transcript(transcript_path, openface_path, output_dir=None, segment_duration=30):
    """
    Cắt file OpenFace thành các đoạn 30s dựa trên khoảng thời gian từ file Transcript.
    
    Parameters:
    -----------
    transcript_path : str
        Đường dẫn đến file CSV Transcript (chứa Start_Time và End_Time)
    openface_path : str
        Đường dẫn đến file CSV OpenFace (chứa cột timestamp)
    output_dir : str, optional
        Thư mục lưu các file output. Nếu None, tạo thư mục mới tên 'output_segments'
    segment_duration : int, optional
        Độ dài mỗi đoạn (giây), mặc định 30s
    
    Returns:
    --------
    list
        Danh sách đường dẫn các file đã tạo
    """
    
    # Lấy prefix từ tên file transcript (ví dụ: 300_Transcript.csv -> 300)
    transcript_filename = os.path.basename(transcript_path)
    file_prefix = transcript_filename.split('_')[0]
    print(f"File prefix: {file_prefix}")
    
    # Đọc file transcript
    transcript_df = pd.read_csv(transcript_path)
    
    # Lấy Start_Time đầu tiên và End_Time cuối cùng
    start_time = transcript_df['Start_Time'].min()
    end_time = transcript_df['End_Time'].max()
    
    print(f"Khoảng thời gian từ Transcript: {start_time}s - {end_time}s")
    
    # Đọc file OpenFace
    openface_df = pd.read_csv(openface_path)
    
    # Lọc dữ liệu OpenFace theo khoảng thời gian từ Transcript
    filtered_df = openface_df[
        (openface_df['timestamp'] >= start_time) & 
        (openface_df['timestamp'] <= end_time)
    ].copy()
    
    print(f"Số dòng OpenFace sau khi lọc: {len(filtered_df)}")
    
    # Tạo thư mục output nếu chưa có
    if output_dir is None:
        output_dir = 'output_segments'
    os.makedirs(output_dir, exist_ok=True)
    
    # Cắt thành các đoạn 30s
    output_files = []
    segment_start = start_time
    segment_index = 1
    
    while segment_start < end_time:
        segment_end = segment_start + segment_duration
        
        # Bỏ qua segment cuối nếu ngắn hơn 30s
        if segment_end > end_time:
            remaining = end_time - segment_start
            if remaining < segment_duration:
                print(f"Bỏ qua segment cuối ({remaining:.1f}s < {segment_duration}s)")
                break
            segment_end = end_time
        
        # Lọc dữ liệu cho đoạn hiện tại
        segment_df = filtered_df[
            (filtered_df['timestamp'] >= segment_start) & 
            (filtered_df['timestamp'] < segment_end)
        ].copy()
        
        # Lưu số hàng ban đầu
        original_rows = len(segment_df)
        
        # Lọc bỏ các hàng có success = 0 hoặc confidence < 0.9
        segment_df = segment_df[
            (segment_df['success'] == 1) & 
            (segment_df['confidence'] >= 0.9)
        ]
        
        # Tính số hàng bị loại bỏ
        removed_rows = original_rows - len(segment_df)
        
        # Bỏ qua segment nếu có nhiều hơn 30 hàng bị loại
        if removed_rows > 30:
            print(f"Bỏ qua segment {segment_index:03d} (loại bỏ {removed_rows} hàng > 30)")
            segment_start = segment_end
            segment_index += 1
            continue
        
        # Lưu file nếu đoạn có dữ liệu
        if len(segment_df) > 0:
            # Tạo tên file với format: 300_segment_001.csv
            output_filename = f"{file_prefix}_segment_{segment_index:03d}.csv"
            output_path = os.path.join(output_dir, output_filename)
            
            segment_df.to_csv(output_path, index=False)
            output_files.append(output_path)
            
            print(f"Đã tạo: {output_filename} ({len(segment_df)} hàng, loại bỏ {removed_rows} hàng)")
        
        segment_start = segment_end
        segment_index += 1
    
    print(f"\nĐã tạo tổng cộng {len(output_files)} file segment")
    return output_files


def batch_process_files(transcript_dir, openface_dir, output_base_dir, start_id=300, end_id=716, segment_duration=30):
    """
    Xử lý hàng loạt file từ start_id đến end_id.
    
    Parameters:
    -----------
    transcript_dir : str
        Thư mục chứa các file Transcript
    openface_dir : str
        Thư mục chứa các file OpenFace
    output_base_dir : str
        Thư mục gốc để lưu output
    start_id : int
        ID bắt đầu (mặc định 300)
    end_id : int
        ID kết thúc (mặc định 716)
    segment_duration : int
        Độ dài mỗi segment (giây)
    
    Returns:
    --------
    dict
        Dictionary với key là file_id và value là danh sách file segments đã tạo
    """
    
    results = {}
    processed_count = 0
    skipped_count = 0
    
    print(f"Bắt đầu xử lý từ ID {start_id} đến {end_id}\n")
    print("=" * 80)
    
    for file_id in range(start_id, end_id + 1):
        # Tạo đường dẫn file
        transcript_path = os.path.join(transcript_dir, f"{file_id}_Transcript.csv")
        openface_path = os.path.join(openface_dir, f"{file_id}_OpenFace2.1.0_Pose_gaze_AUs.csv")
        
        # Kiểm tra cả 2 file có tồn tại không
        if not os.path.exists(transcript_path):
            print(f"⊘ Bỏ qua {file_id}: Không tìm thấy file Transcript")
            skipped_count += 1
            continue
            
        if not os.path.exists(openface_path):
            print(f"⊘ Bỏ qua {file_id}: Không tìm thấy file OpenFace")
            skipped_count += 1
            continue
        
        # Tạo thư mục output cho file này (VD: 300_P)
        output_dir = os.path.join(output_base_dir, f"{file_id}_P")
        
        print(f"\n► Đang xử lý ID {file_id}...")
        
        try:
            # Gọi hàm xử lý
            output_files = split_openface_by_transcript(
                transcript_path=transcript_path,
                openface_path=openface_path,
                output_dir=output_dir,
                segment_duration=segment_duration
            )
            
            results[file_id] = output_files
            processed_count += 1
            print(f"✓ Hoàn thành ID {file_id}: {len(output_files)} segments")
            
        except Exception as e:
            print(f"✗ Lỗi khi xử lý ID {file_id}: {str(e)}")
            skipped_count += 1
            continue
    
    print("\n" + "=" * 80)
    print(f"\nTÓM TẮT:")
    print(f"  • Đã xử lý thành công: {processed_count} files")
    print(f"  • Đã bỏ qua: {skipped_count} files")
    print(f"  • Tổng segments tạo ra: {sum(len(files) for files in results.values())}")
    
    return results


# Ví dụ sử dụng batch processing
if __name__ == "__main__":
    # Cấu hình đường dẫn
    TRANSCRIPT_DIR = "/kaggle/input/edaic-transcript"
    OPENFACE_DIR = "/kaggle/input/edaic-aus-pose-gaze"
    OUTPUT_BASE_DIR = "/kaggle/working"
    
    # Chạy batch processing
    results = batch_process_files(
        transcript_dir=TRANSCRIPT_DIR,
        openface_dir=OPENFACE_DIR,
        output_base_dir=OUTPUT_BASE_DIR,
        start_id=300,
        end_id=716,
        segment_duration=30
    )
    
    # In kết quả chi tiết (tùy chọn)
    print("\n" + "=" * 80)
    print("CHI TIẾT KẾT QUẢ:")
    for file_id, files in sorted(results.items()):
        print(f"  {file_id}_P: {len(files)} segments")

Bắt đầu xử lý từ ID 300 đến 716


► Đang xử lý ID 300...
File prefix: 300
Khoảng thời gian từ Transcript: 14.3s - 648.3s
Số dòng OpenFace sau khi lọc: 19021
Bỏ qua segment 001 (loại bỏ 67 hàng > 30)
Đã tạo: 300_segment_002.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_003.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_004.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_005.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_006.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_007.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_008.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_009.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_010.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_011.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_012.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_013.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_014.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_015.csv (900 hàng, loại bỏ 0 hàng)
Đã tạo: 300_segment_016.csv (89

In [3]:
import os
import shutil

# Thư mục gốc chứa các folder cần nén
base_dir = "/kaggle/working"

# Lặp qua tất cả các thư mục con trong base_dir
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    
    # Chỉ xử lý nếu là thư mục (bỏ qua file)
    if os.path.isdir(folder_path):
        zip_path = os.path.join(base_dir, folder_name)  # Không cần .zip, shutil tự thêm
        try:
            shutil.make_archive(zip_path, 'zip', folder_path)
            print(f"✅ Đã nén: {folder_name}.zip")
        except Exception as e:
            print(f"⚠️ Lỗi khi nén {folder_name}: {e}")

✅ Đã nén: 464_P.zip
✅ Đã nén: 389_P.zip
✅ Đã nén: 343_P.zip
✅ Đã nén: 309_P.zip
✅ Đã nén: 423_P.zip
✅ Đã nén: 355_P.zip
✅ Đã nén: 461_P.zip
✅ Đã nén: 408_P.zip
✅ Đã nén: 473_P.zip
✅ Đã nén: 425_P.zip
✅ Đã nén: 465_P.zip
✅ Đã nén: 358_P.zip
✅ Đã nén: 305_P.zip
✅ Đã nén: 677_P.zip
✅ Đã nén: 414_P.zip
✅ Đã nén: 421_P.zip
✅ Đã nén: 383_P.zip
✅ Đã nén: 371_P.zip
✅ Đã nén: 454_P.zip
✅ Đã nén: 491_P.zip
✅ Đã nén: 406_P.zip
✅ Đã nén: 604_P.zip
✅ Đã nén: 370_P.zip
✅ Đã nén: 483_P.zip
✅ Đã nén: 471_P.zip
✅ Đã nén: 480_P.zip
✅ Đã nén: 640_P.zip
✅ Đã nén: 467_P.zip
✅ Đã nén: 350_P.zip
✅ Đã nén: 365_P.zip
✅ Đã nén: 432_P.zip
✅ Đã nén: 325_P.zip
✅ Đã nén: 399_P.zip
✅ Đã nén: 307_P.zip
✅ Đã nén: 351_P.zip
✅ Đã nén: 361_P.zip
✅ Đã nén: 440_P.zip
✅ Đã nén: 354_P.zip
✅ Đã nén: 439_P.zip
✅ Đã nén: 436_P.zip
✅ Đã nén: 479_P.zip
✅ Đã nén: 477_P.zip
✅ Đã nén: 459_P.zip
✅ Đã nén: 698_P.zip
✅ Đã nén: 340_P.zip
✅ Đã nén: 372_P.zip
✅ Đã nén: 661_P.zip
✅ Đã nén: 316_P.zip
✅ Đã nén: 633_P.zip
✅ Đã nén: 359_P.zip
